[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/danpele/Time-Series-Analysis/blob/main/chapter9_lecture_notebook.ipynb)

---

# Chapter 9: Prophet & TBATS for Multiple Seasonality

**Course:** Time Series Analysis and Forecasting  
**Program:** Bachelor program, Faculty of Cybernetics, Statistics and Economic Informatics, Bucharest University of Economic Studies, Romania  
**Academic Year:** 2025-2026

---

## Learning Objectives

By the end of this notebook, you will be able to:
1. Understand why standard SARIMA fails for multiple seasonality
2. Apply TBATS for complex seasonal patterns
3. Use Facebook Prophet for forecasting with holidays
4. Compare performance of Prophet vs TBATS
5. Handle multiple seasonal periods (daily, weekly, annual)
6. Interpret model components and diagnostics

## Setup and Imports

In [ ]:
# Install required packages (uncomment if needed in Colab)
# !pip install prophet tbats statsmodels pandas numpy matplotlib -q

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# Statistical models
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# Prophet
try:
    from prophet import Prophet
    HAS_PROPHET = True
except ImportError:
    HAS_PROPHET = False
    print("Prophet not installed. Install with: pip install prophet")

# TBATS
try:
    from tbats import TBATS
    HAS_TBATS = True
except ImportError:
    HAS_TBATS = False
    print("TBATS not installed. Install with: pip install tbats")

from sklearn.metrics import mean_squared_error, mean_absolute_error

# Plotting style
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.facecolor'] = 'none'
plt.rcParams['figure.facecolor'] = 'none'
plt.rcParams['axes.grid'] = False
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['legend.frameon'] = False

COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666'}

print("Setup complete!")
print(f"Prophet available: {HAS_PROPHET}")
print(f"TBATS available: {HAS_TBATS}")

## 1. The Multiple Seasonality Challenge

### Why SARIMA Fails

Standard SARIMA can only handle **one** seasonal period (parameter $m$).

Many real-world time series have **multiple** overlapping seasonal patterns:
- **Hourly electricity demand**: daily (24h) + weekly (168h) + annual (8760h)
- **Retail sales**: weekly + monthly + annual
- **Web traffic**: daily + weekly + holiday effects

In [ ]:
# Generate synthetic data with multiple seasonality
np.random.seed(42)

# Create hourly data for 2 years
n_hours = 24 * 365 * 2  # 2 years of hourly data
dates = pd.date_range(start='2022-01-01', periods=n_hours, freq='H')

# Components
t = np.arange(n_hours)

# Trend (slight upward)
trend = 100 + 0.001 * t

# Daily seasonality (24-hour cycle) - peak at 6pm
daily = 20 * np.sin(2 * np.pi * t / 24 - np.pi/2)

# Weekly seasonality (168-hour cycle) - lower on weekends
weekly = 15 * np.sin(2 * np.pi * t / 168)

# Annual seasonality (8760-hour cycle) - peak in winter
annual = 30 * np.cos(2 * np.pi * t / 8760)

# Combine with noise
noise = np.random.normal(0, 5, n_hours)
y = trend + daily + weekly + annual + noise

# Create DataFrame
df = pd.DataFrame({'ds': dates, 'y': y})
df.set_index('ds', inplace=True)

print("Synthetic Energy Demand Data")
print("="*50)
print(f"Period: {df.index[0]} to {df.index[-1]}")
print(f"Observations: {len(df):,} hours")
print(f"\nSeasonalities:")
print(f"  - Daily: 24 hours")
print(f"  - Weekly: 168 hours")
print(f"  - Annual: 8,760 hours")

In [ ]:
# Visualize the multiple seasonalities
fig, axes = plt.subplots(3, 1, figsize=(14, 10))

# Full series (subsample for visibility)
subsample = df.iloc[::24]  # Daily averages
axes[0].plot(subsample.index, subsample['y'], color=COLORS['blue'], linewidth=0.5)
axes[0].set_title('Energy Demand: Full Series (daily averages)', fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Demand')

# One week (168 hours) - shows daily pattern
one_week = df.iloc[:168]
axes[1].plot(one_week.index, one_week['y'], color=COLORS['orange'], linewidth=1)
axes[1].set_title('One Week: Daily Pattern Visible', fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Demand')

# One day (24 hours) - shows hourly pattern
one_day = df.iloc[:24]
axes[2].plot(range(24), one_day['y'], color=COLORS['green'], linewidth=2, marker='o')
axes[2].set_title('One Day: Hourly Pattern', fontweight='bold')
axes[2].set_xlabel('Hour')
axes[2].set_ylabel('Demand')
axes[2].set_xticks(range(0, 24, 3))

plt.tight_layout()
plt.show()

print("Key insight: Standard SARIMA with m=24 can't capture weekly/annual patterns!")

## 2. TBATS: Trigonometric Seasonality

### What TBATS Stands For

- **T**rigonometric: Fourier terms for seasonality
- **B**ox-Cox: Variance stabilization
- **A**RMA: Error autocorrelation
- **T**rend: Damped local linear trend
- **S**easonal: Multiple seasonal periods

### Fourier Representation of Seasonality

$$s_t^{(i)} = \sum_{j=1}^{k_i} \left[ a_j^{(i)} \cos\left(\frac{2\pi j t}{m_i}\right) + b_j^{(i)} \sin\left(\frac{2\pi j t}{m_i}\right) \right]$$

Where:
- $m_i$ = seasonal period $i$
- $k_i$ = number of harmonics (complexity of shape)
- Maximum $k_i = m_i / 2$

In [ ]:
# Demonstrate Fourier terms for seasonality
t = np.linspace(0, 2*np.pi, 100)

fig, axes = plt.subplots(2, 2, figsize=(14, 8))

# K=1: Simple sine wave
k1 = np.sin(t)
axes[0, 0].plot(t, k1, color=COLORS['blue'], linewidth=2)
axes[0, 0].set_title('K=1: Simple Sinusoid', fontweight='bold')
axes[0, 0].set_xlabel('Time')

# K=2: Can capture asymmetry
k2 = np.sin(t) + 0.5*np.sin(2*t)
axes[0, 1].plot(t, k2, color=COLORS['orange'], linewidth=2)
axes[0, 1].set_title('K=2: More Flexible (asymmetric peaks)', fontweight='bold')
axes[0, 1].set_xlabel('Time')

# K=5: Complex patterns
k5 = np.sin(t) + 0.5*np.sin(2*t) + 0.3*np.sin(3*t) + 0.2*np.sin(4*t) + 0.1*np.sin(5*t)
axes[1, 0].plot(t, k5, color=COLORS['green'], linewidth=2)
axes[1, 0].set_title('K=5: Complex Pattern (sharp features)', fontweight='bold')
axes[1, 0].set_xlabel('Time')

# Comparison
axes[1, 1].plot(t, k1, label='K=1', linewidth=2)
axes[1, 1].plot(t, k2, label='K=2', linewidth=2)
axes[1, 1].plot(t, k5, label='K=5', linewidth=2)
axes[1, 1].set_title('Comparison: More K = More Flexibility', fontweight='bold')
axes[1, 1].set_xlabel('Time')
axes[1, 1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)

plt.tight_layout()
plt.subplots_adjust(bottom=0.12)
plt.show()

print("Key: K harmonics = K pairs of (sin, cos) terms")
print("More K = more complex seasonal shapes (but risk of overfitting)")

In [ ]:
if HAS_TBATS:
    # Prepare data for TBATS (use daily data for faster computation)
    df_daily = df.resample('D').mean()
    
    # Train/test split
    train_size = int(len(df_daily) * 0.8)
    train = df_daily.iloc[:train_size]
    test = df_daily.iloc[train_size:]
    
    print("TBATS Model Training")
    print("="*50)
    print(f"Training period: {train.index[0].date()} to {train.index[-1].date()}")
    print(f"Test period: {test.index[0].date()} to {test.index[-1].date()}")
    print(f"\nFitting TBATS with seasonal periods: [7, 365]...")
    
    # Fit TBATS with weekly and annual seasonality
    estimator = TBATS(seasonal_periods=[7, 365.25])
    model_tbats = estimator.fit(train['y'].values)
    
    print("\nModel Summary:")
    print(model_tbats.summary())
else:
    print("TBATS not available. Please install with: pip install tbats")

In [ ]:
if HAS_TBATS:
    # Forecast
    forecast_horizon = len(test)
    forecast_tbats = model_tbats.forecast(steps=forecast_horizon)
    
    # Metrics
    rmse_tbats = np.sqrt(mean_squared_error(test['y'], forecast_tbats))
    mae_tbats = mean_absolute_error(test['y'], forecast_tbats)
    mape_tbats = np.mean(np.abs((test['y'].values - forecast_tbats) / test['y'].values)) * 100
    
    print("TBATS Forecast Results")
    print("="*50)
    print(f"RMSE: {rmse_tbats:.2f}")
    print(f"MAE:  {mae_tbats:.2f}")
    print(f"MAPE: {mape_tbats:.1f}%")
    
    # Plot
    fig, ax = plt.subplots(figsize=(14, 5))
    
    ax.plot(train.index[-60:], train['y'].iloc[-60:], color=COLORS['blue'], label='Train', linewidth=1)
    ax.plot(test.index, test['y'], color=COLORS['blue'], label='Actual', linewidth=1)
    ax.plot(test.index, forecast_tbats, color=COLORS['red'], label='TBATS Forecast', linewidth=1.5, linestyle='--')
    ax.axvline(x=test.index[0], color='black', linestyle=':', alpha=0.5)
    ax.set_title('TBATS Forecast vs Actual', fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Demand')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=3, frameon=False)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.18)
    plt.show()

## 3. Prophet: Facebook's Forecasting Tool

### Prophet Decomposition

Prophet uses an additive decomposition:

$$y(t) = g(t) + s(t) + h(t) + \varepsilon_t$$

Where:
- $g(t)$ = **Trend** (linear or logistic growth with changepoints)
- $s(t)$ = **Seasonality** (Fourier series)
- $h(t)$ = **Holidays** (user-specified events)
- $\varepsilon_t$ = Error term

### Key Features

1. **Automatic changepoint detection** in trend
2. **Multiple seasonalities** (daily, weekly, annual)
3. **Holiday effects** (easily incorporated)
4. **Robust to missing data**
5. **Intuitive parameters** for analysts

In [ ]:
if HAS_PROPHET:
    # Prepare data for Prophet (requires 'ds' and 'y' columns)
    df_prophet = df_daily.reset_index()
    df_prophet.columns = ['ds', 'y']
    
    # Train/test split
    train_prophet = df_prophet.iloc[:train_size]
    test_prophet = df_prophet.iloc[train_size:]
    
    print("Prophet Model Training")
    print("="*50)
    
    # Initialize Prophet model
    model_prophet = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,  # Using daily aggregated data
        changepoint_prior_scale=0.05,  # Flexibility of trend changepoints
        seasonality_prior_scale=10,    # Strength of seasonality
    )
    
    # Fit model
    model_prophet.fit(train_prophet)
    print("Model fitted successfully!")
else:
    print("Prophet not available. Please install with: pip install prophet")

In [ ]:
if HAS_PROPHET:
    # Create future dataframe for prediction
    future = model_prophet.make_future_dataframe(periods=len(test_prophet))
    
    # Forecast
    forecast_prophet = model_prophet.predict(future)
    
    # Extract test predictions
    test_predictions = forecast_prophet.iloc[train_size:]['yhat'].values
    
    # Metrics
    rmse_prophet = np.sqrt(mean_squared_error(test['y'], test_predictions))
    mae_prophet = mean_absolute_error(test['y'], test_predictions)
    mape_prophet = np.mean(np.abs((test['y'].values - test_predictions) / test['y'].values)) * 100
    
    print("Prophet Forecast Results")
    print("="*50)
    print(f"RMSE: {rmse_prophet:.2f}")
    print(f"MAE:  {mae_prophet:.2f}")
    print(f"MAPE: {mape_prophet:.1f}%")

In [ ]:
if HAS_PROPHET:
    # Plot Prophet components
    fig = model_prophet.plot_components(forecast_prophet)
    plt.suptitle('Prophet Component Decomposition', fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()
    
    print("\nComponent Interpretation:")
    print("- Trend: Shows overall direction with changepoints")
    print("- Weekly: Day-of-week effects (weekday vs weekend)")
    print("- Yearly: Annual seasonal pattern (summer vs winter)")

In [ ]:
if HAS_PROPHET:
    # Forecast plot with uncertainty
    fig, ax = plt.subplots(figsize=(14, 5))
    
    # Historical data
    ax.plot(train_prophet['ds'].iloc[-60:], train_prophet['y'].iloc[-60:], 
            color=COLORS['blue'], label='Train', linewidth=1)
    ax.plot(test_prophet['ds'], test_prophet['y'], 
            color=COLORS['blue'], label='Actual', linewidth=1)
    
    # Forecast
    test_forecast = forecast_prophet.iloc[train_size:]
    ax.plot(test_forecast['ds'], test_forecast['yhat'], 
            color=COLORS['red'], label='Prophet Forecast', linewidth=1.5, linestyle='--')
    
    # Uncertainty interval
    ax.fill_between(test_forecast['ds'], 
                    test_forecast['yhat_lower'], 
                    test_forecast['yhat_upper'],
                    color=COLORS['red'], alpha=0.2, label='95% CI')
    
    ax.axvline(x=test_prophet['ds'].iloc[0], color='black', linestyle=':', alpha=0.5)
    ax.set_title('Prophet Forecast with Uncertainty Interval', fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Demand')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=4, frameon=False)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.18)
    plt.show()

## 4. Adding Holiday Effects in Prophet

Prophet makes it easy to incorporate known events/holidays.

In [ ]:
if HAS_PROPHET:
    # Define holidays
    holidays = pd.DataFrame({
        'holiday': 'major_holiday',
        'ds': pd.to_datetime(['2022-01-01', '2022-12-25', '2022-07-04',
                             '2023-01-01', '2023-12-25', '2023-07-04']),
        'lower_window': -1,  # Day before
        'upper_window': 1,   # Day after
    })
    
    print("Holiday DataFrame")
    print(holidays)
    
    # Fit model with holidays
    model_with_holidays = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        holidays=holidays
    )
    
    model_with_holidays.fit(train_prophet)
    
    # Forecast
    future_h = model_with_holidays.make_future_dataframe(periods=len(test_prophet))
    forecast_h = model_with_holidays.predict(future_h)
    
    print("\nModel with holidays fitted successfully!")
    print("Holiday effect will appear in component decomposition.")

## 5. Changepoints in Prophet

Prophet automatically detects points where the trend changes slope.

In [ ]:
if HAS_PROPHET:
    # Get changepoints
    changepoints = model_prophet.changepoints
    
    print(f"Detected {len(changepoints)} changepoints:")
    print(changepoints[:5].tolist())  # Show first 5
    
    # Plot trend with changepoints
    fig, ax = plt.subplots(figsize=(14, 5))
    
    ax.plot(forecast_prophet['ds'], forecast_prophet['trend'], 
            color=COLORS['blue'], linewidth=2, label='Trend')
    
    for cp in changepoints:
        ax.axvline(x=cp, color=COLORS['red'], linestyle='--', alpha=0.3)
    
    ax.set_title('Prophet Trend with Changepoints', fontweight='bold')
    ax.set_xlabel('Date')
    ax.set_ylabel('Trend Value')
    ax.legend(['Trend', 'Changepoints'], loc='upper center', 
              bbox_to_anchor=(0.5, -0.12), ncol=2, frameon=False)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.18)
    plt.show()

## 6. Prophet vs TBATS Comparison

In [ ]:
if HAS_PROPHET and HAS_TBATS:
    # Compare forecasts
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Forecasts comparison
    axes[0].plot(test.index, test['y'], color=COLORS['blue'], label='Actual', linewidth=1)
    axes[0].plot(test.index, forecast_tbats, color=COLORS['orange'], 
                 label=f'TBATS (RMSE={rmse_tbats:.1f})', linewidth=1.5, linestyle='--')
    axes[0].plot(test.index, test_predictions, color=COLORS['green'], 
                 label=f'Prophet (RMSE={rmse_prophet:.1f})', linewidth=1.5, linestyle=':')
    axes[0].set_title('Forecast Comparison', fontweight='bold')
    axes[0].set_xlabel('Date')
    axes[0].set_ylabel('Demand')
    axes[0].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False)
    
    # Metrics comparison
    metrics = ['RMSE', 'MAE', 'MAPE (%)']
    tbats_vals = [rmse_tbats, mae_tbats, mape_tbats]
    prophet_vals = [rmse_prophet, mae_prophet, mape_prophet]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    bars1 = axes[1].bar(x - width/2, tbats_vals, width, label='TBATS', color=COLORS['orange'])
    bars2 = axes[1].bar(x + width/2, prophet_vals, width, label='Prophet', color=COLORS['green'])
    
    axes[1].set_ylabel('Value')
    axes[1].set_title('Model Performance Comparison', fontweight='bold')
    axes[1].set_xticks(x)
    axes[1].set_xticklabels(metrics)
    axes[1].legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=2, frameon=False)
    
    # Add value labels
    for bar, val in zip(bars1, tbats_vals):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                     f'{val:.1f}', ha='center', fontsize=9)
    for bar, val in zip(bars2, prophet_vals):
        axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                     f'{val:.1f}', ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.subplots_adjust(bottom=0.2)
    plt.show()
    
    print("\nModel Comparison Summary")
    print("="*50)
    print(f"{'Metric':<10} {'TBATS':>10} {'Prophet':>10}")
    print("-"*30)
    print(f"{'RMSE':<10} {rmse_tbats:>10.2f} {rmse_prophet:>10.2f}")
    print(f"{'MAE':<10} {mae_tbats:>10.2f} {mae_prophet:>10.2f}")
    print(f"{'MAPE (%)':<10} {mape_tbats:>10.1f} {mape_prophet:>10.1f}")

## 7. Multiplicative vs Additive Seasonality

Prophet supports both modes:
- **Additive**: Seasonal effect is constant ($y = trend + season$)
- **Multiplicative**: Seasonal effect scales with level ($y = trend \times (1 + season)$)

In [ ]:
# Demonstrate additive vs multiplicative
np.random.seed(42)
t = np.arange(365 * 2)

# Trend
trend = 100 + 0.2 * t

# Seasonal pattern
seasonal = 20 * np.sin(2 * np.pi * t / 365)

# Additive: constant seasonal amplitude
y_additive = trend + seasonal + np.random.normal(0, 5, len(t))

# Multiplicative: seasonal amplitude grows with trend
y_multiplicative = trend * (1 + 0.2 * np.sin(2 * np.pi * t / 365)) + np.random.normal(0, 5, len(t))

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(t, y_additive, color=COLORS['blue'], linewidth=0.8)
axes[0].plot(t, trend, color=COLORS['red'], linewidth=2, linestyle='--', label='Trend')
axes[0].set_title('Additive Seasonality\n(Constant amplitude)', fontweight='bold')
axes[0].set_xlabel('Day')
axes[0].set_ylabel('Value')
axes[0].legend()

axes[1].plot(t, y_multiplicative, color=COLORS['orange'], linewidth=0.8)
axes[1].plot(t, trend, color=COLORS['red'], linewidth=2, linestyle='--', label='Trend')
axes[1].set_title('Multiplicative Seasonality\n(Amplitude grows with level)', fontweight='bold')
axes[1].set_xlabel('Day')
axes[1].set_ylabel('Value')
axes[1].legend()

plt.tight_layout()
plt.show()

print("When to use each mode:")
print("- Additive: Seasonal variation is constant (e.g., temperature)")
print("- Multiplicative: Seasonal variation grows with level (e.g., retail sales)")

In [ ]:
if HAS_PROPHET:
    # Fit Prophet with multiplicative seasonality
    df_mult = pd.DataFrame({
        'ds': pd.date_range(start='2022-01-01', periods=len(y_multiplicative), freq='D'),
        'y': y_multiplicative
    })
    
    model_mult = Prophet(
        seasonality_mode='multiplicative',
        yearly_seasonality=True,
        weekly_seasonality=False,
        daily_seasonality=False
    )
    
    model_mult.fit(df_mult)
    future_mult = model_mult.make_future_dataframe(periods=30)
    forecast_mult = model_mult.predict(future_mult)
    
    # Plot components
    fig = model_mult.plot_components(forecast_mult)
    plt.suptitle('Prophet Components (Multiplicative Mode)', fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.show()

## Summary

### Key Takeaways

1. **Multiple Seasonality** is common in real data but SARIMA can only handle one period

2. **TBATS** (Trigonometric, Box-Cox, ARMA, Trend, Seasonal):
   - Uses Fourier terms for flexible seasonal patterns
   - Automatic selection of harmonics via AIC
   - State-space formulation for efficient computation
   - Good for automatic forecasting

3. **Prophet**:
   - Decomposable model: $y = g(t) + s(t) + h(t) + \varepsilon$
   - Easy incorporation of holidays and events
   - Automatic changepoint detection
   - Interpretable components
   - Good when domain knowledge is available

### When to Use What?

| Situation | Recommendation |
|-----------|----------------|
| Known holidays/events | Prophet |
| Automatic forecasting, no domain knowledge | TBATS |
| Need interpretable components | Prophet |
| Very long seasonal periods (e.g., 8760 hours) | TBATS (more efficient) |
| Need uncertainty intervals easily | Prophet |
| Business reporting & communication | Prophet (cleaner plots) |